#### Real World application 2

In [1]:
import pymongo
import json

client = pymongo.MongoClient('localhost', 27017)
db = client["Homeworks"]
collection = db["authors"]

In [2]:
with open('authors.json', "r") as f:
    data = json.load(f)

for doc in data:
    if '_id' in doc:
        doc['_id'] = str(doc['_id']['$oid'])

collection.insert_many(data)

InsertManyResult(['6242c7c5022d5c5d474a044e', '6242c7c5022d5c5d474a044f', '6242c7c5022d5c5d474a0450', '6242c7c5022d5c5d474a0451', '6242c7c5022d5c5d474a0452', '6242c7c5022d5c5d474a0453', '6242c7c5022d5c5d474a0454', '6242c7c5022d5c5d474a0455', '6242c7c5022d5c5d474a0456', '6242c7c5022d5c5d474a0457', '6242c7c5022d5c5d474a0458', '6242c7c5022d5c5d474a0459', '6242c7c5022d5c5d474a045a', '6242c7c5022d5c5d474a045b', '6242c7c5022d5c5d474a045c', '6242c7c5022d5c5d474a045d', '6242c7c5022d5c5d474a045e', '6242c7c5022d5c5d474a045f', '6242c7c5022d5c5d474a0460', '6242c7c5022d5c5d474a0461', '6242c7c5022d5c5d474a0462', '6242c7c5022d5c5d474a0463', '6242c7c5022d5c5d474a0464', '6242c7c5022d5c5d474a0465', '6242c7c5022d5c5d474a0466', '6242c7c5022d5c5d474a0467', '6242c7c5022d5c5d474a0468', '6242c7c5022d5c5d474a0469', '6242c7c5022d5c5d474a046a', '6242c7c5022d5c5d474a046b', '6242c7c5022d5c5d474a046c', '6242c7c5022d5c5d474a046d', '6242c7c5022d5c5d474a046e', '6242c7c5022d5c5d474a046f', '6242c7c5022d5c5d474a0470', '6

In [4]:
# 1. Create an index, explain your choice of key.
''' _id est une valeur unique pour chaque document, c'est plus facile de faire les recherches par _id.'''

collection.create_index([("_id",1) ])

'_id_1'

In [9]:
# 2. What is the average length of "pmid_list"
pipeline = [
    {"$unwind": "$pmid_list"},
    {"$group": {"_id": None, "avg_length": 
                {"$avg": 
                 {"$strLenCP": 
                  {"$toString": "$pmid_list"}}}}}] # d'abord on transforme pmid_list en string et après on calcule la longueur

result = list(collection.aggregate(pipeline))
print(result[0]['avg_length'])

7.744977920319982


In [11]:
# 3. How many distinct affiliations are there ?
pipeline = [
    {"$unwind": "$oa04_affiliations"}, 
    {"$group": {"_id": "$oa04_affiliations.Affiliation"}},  # group by affiliation
    {"$count": "num_unique_affiliations"} 
]

result = list(collection.aggregate(pipeline))
print(result)

[{'num_unique_affiliations': 318595}]


In [21]:
# 4.Find authors with atleast one "COM" AffiliationType

pipeline = [{"$unwind": "$oa04_affiliations"},
            {"$match": {"oa04_affiliations.AffiliationType": "COM"}},
            {"$project": {"_id": 1, "name": 1, "oa04_affiliations": 1}}]

result = list(collection.aggregate(pipeline))

authors_com = []
for author in result:
    authors_com.append(author['_id'])

In [24]:
# 5.How many authors switched the AffiliationType ?

pipeline = [
    {"$unwind": "$oa04_affiliations"},
    {"$group": {"_id": "$_id", "num_types": {"$addToSet": "$oa04_affiliations.AffiliationType"}}},
    {"$project": {"_id": 1, "num_types": {"$size": "$num_types"}}},
    {"$match": {"num_types": {"$gt": 1}}},
    {"$count": "num_authors"}]

result = list(collection.aggregate(pipeline))
print(result)

[{'num_authors': 21670}]


In [36]:
# 6. Find affiliation with the word "China"

pipeline = [{"$unwind": "$oa04_affiliations"},
    {"$match": {"oa04_affiliations.Affiliation": {"$regex": "China"}}},
    {"$project": {"_id": 1, "name": 1, "oa04_affiliations.Affiliation": 1}}]

result = list(collection.aggregate(pipeline))

In [37]:
# 7. Get the pmids of papers published in 2019

pipeline = [
    {"$unwind": "$more_info"},
    {"$match": {"more_info.PubYear": 2019}},
    {"$project": {"_id": 1, "name": 1, "more_info.PMID": 1}}]

result = list(collection.aggregate(pipeline))
result[0] # exemple de résultat


{'_id': '6242c7c5022d5c5d474a044e', 'more_info': {'PMID': 30972486}}

In [2]:
# 8. Count the number of doc with "oa06_researcher_education" OR "oa04_affiliations" key and 
# with the "oa06_researcher_education" AND "oa04_affiliations" .

pipeline1 = [{"$match": {"$or": [{"oa06_researcher_education": {"$exists": True}}, 
                                {"oa04_affiliations": {"$exists": True}}]}}]

pipeline2 =[{"$match": {"$and": [{"oa06_researcher_education": {"$exists": True}},
                                {"oa04_affiliations": {"$exists": True}}]}}]

                               
result1= list(collection.aggregate(pipeline1))
print(len(result1))

result2= list(collection.aggregate(pipeline2))
print(len(result2))

53086
903


In [24]:
# 9. What's the average "BeginYear", "BeginYear" is the type string, of "oa06_researcher_education".
pipeline = [{"$unwind": "$oa06_researcher_education"},
    {"$addFields": {"BeginYear": {"$cond": {"if": {"$eq": ["$oa06_researcher_education.BeginYear",' ']}, 
                                            "then":None, "else": {"$toInt": "$oa06_researcher_education.BeginYear"}}}}},
    {"$match": {"BeginYear": {"$ne":None}}},
    {"$group": {"_id": None, "avg_BeginYear": {"$avg": "$BeginYear"}}}]

result = list(collection.aggregate(pipeline))
print(result[0]['avg_BeginYear'])


2003.8455284552845


In [25]:
# 10.Count the distinct country of "oa06_researcher_education"

pipeline = [{"$unwind": "$oa06_researcher_education"},
    {"$group": {"_id": "$oa06_researcher_education.Country"}},
    {"$count": "num_unique_countries"}]

result = list(collection.aggregate(pipeline))
print(result)

[{'num_unique_countries': 101}]


In [ ]:
# 11. Does the length of pmid_list and more_info always match ?

???

In [ ]:
# 12. Does the length of pmid_list and "oa04_affiliations" always match ?